In [1]:
import faiss                   # make faiss available
import numpy as np
from os import listdir
from os.path import isfile, join
import pickle
import json
import hashlib
import pandas as pd
import sys
sys.path.insert(1, '../utils/')
from embeddings import vectorize
from text_utils import encode
import redis

pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
redis_docs = redis.Redis(connection_pool=pool)

In [2]:
d = 1536  

In [3]:
faiss_index_path = '/Users/vli/Work/RetailSearch/RetailSearch/deployment/faiss/index/grocery.fi'

In [4]:
search_index = faiss.read_index(faiss_index_path)

In [5]:
doc_id_path = '/Users/vli/Work/RetailSearch/RetailSearch/deployment/faiss/index/grocery.id'

In [6]:
doc_idx = pickle.load(open(doc_id_path, "rb"))

In [7]:
def search(text):
    encode = vectorize(text)
    D, I = search_index.search(np.array([encode]), 200)
    result_ids = doc_idx[I[0]]
    results = redis_docs.mget(result_ids)
    return results